In [1]:
from requests import get # la vamos a usar para solicitar la página mediante su URL 
from bs4 import BeautifulSoup # biblioteca para web scraping
import pandas as pd
import string                              
from nltk.corpus import stopwords # para remover palabras que no agregan significado          
from nltk.stem import PorterStemmer # para llevar las palabras a su forma raíz        
from nltk.tokenize import TweetTokenizer # para transformar cada palabra en un token
from IPython.display import clear_output
from collections import defaultdict
import numpy as np

In [ ]:
import pickle
def save_obj(obj, name):
       with open(name + '.pkl', 'wb') as f:
           pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
       with open(name + '.pkl', 'rb') as f:
           return pickle.load(f)

In [2]:
def etiquetas(palabras,frecuencias,clases):
    import numpy as np
    x = np.zeros((1, 6))
    for palabra in palabras:
        for t in clases:
            x[0,t] += frecuencias.get((palabra, t),0)
    x[0,0]=1
    return x

stop        = stopwords.words('english')
tokenizer   = TweetTokenizer(preserve_case = False, strip_handles = True, reduce_len = True)
stemmer     = PorterStemmer()
webs        = pd.ExcelFile("WEBs.xls")
df          = webs.parse("Categorias")
textos      = pd.DataFrame(columns = [ 'Texto' , 'Clase'])
clases_df   = set(df["CÓDIGO"].value_counts().index)
frecuencias = defaultdict(int)
for i in df.index:
    url       = df["URL"][i]
    pagina    = get(url)
    html      = BeautifulSoup(pagina.text,"html.parser")
    parrafos  = [ x.text.strip() for x in html.findAll("p") if x.text != "\n" ]
    for p in parrafos:
        palabras = p.split()
        if 15 <= len(palabras):
            clear_output( wait = True )
            print("procesando {}".format(url))
            print(f"{i} de {len(df)}")
            texto_tokens = tokenizer.tokenize(p)
            texto_clean  = [ word for word in texto_tokens if word not in stop and word not in string.punctuation ]
            texto_stem   = [ stemmer.stem(word) for word in texto_clean ]
            for t in range(len(texto_stem)//20):
                j = 20 * t
                palabras  = texto_stem[ j : 20 * (t+1) ] 
                textos   = textos.append({'Texto' : palabras, 'Clase' : df["CÓDIGO"][i] }, ignore_index = True)

vocabulario = set([x for m in textos.index for x in textos.loc[m].Texto ])                
clases_df   = set(df["CÓDIGO"].value_counts().index)

for i in textos.index:
    for palabra in textos.loc[i].Texto:
        frecuencias[(palabra, textos.loc[i].Clase)] += 1
        clear_output( wait = True )
        print(f"Creando Frecuencias {i} de {len(textos)}")

for i in clases_df:
    textos[f"Clase{i}"] = textos.Texto.apply(lambda x : etiquetas(x,frecuencias,clases_df)[0,i])
print("dataset terminado")
print(textos.describe())

dataset terminado
             Clase1        Clase2        Clase3        Clase4        Clase5
count  15017.000000  15017.000000  15017.000000  15017.000000  15017.000000
mean     762.425451    821.655391    779.091030    706.930013    749.688486
std      398.578877    386.416180    341.336204    470.000742    579.700295
min       11.000000      8.000000      0.000000      6.000000      1.000000
25%      495.000000    544.000000    532.000000    391.000000    352.000000
50%      683.000000    759.000000    732.000000    572.000000    586.000000
75%      930.000000   1027.000000    972.000000    869.000000    948.000000
max     5122.000000   3158.000000   2799.000000   4928.000000   4390.000000


In [3]:
textos

,Texto,Clase,Clase1,Clase2,Clase3,Clase4,Clase5
0,"[alexand, iii, macedon, greek, αλέξανδρος, γʹ,...",1,1266.0,468.0,460.0,317.0,462.0
1,"[bc, commonli, known, alexand, great, greek, ὁ...",1,2073.0,1007.0,740.0,279.0,522.0
2,"[pella, 356, bc, succeed, father, philip, ii, ...",1,1457.0,1144.0,1206.0,698.0,1348.0
3,"[age, thirti, creat, one, largest, empir, anci...",1,1272.0,1307.0,1032.0,829.0,1088.0
4,"[youth, alexand, tutor, aristotl, age, 16, phi...",1,1497.0,694.0,623.0,485.0,580.0
...,...,...,...,...,...,...,...
15012,"[south, chile, mont, verd, citat, need, americ...",5,534.0,528.0,552.0,618.0,1115.0
15013,"[common, style, stone, tool, product, make, kn...",5,703.0,650.0,594.0,370.0,2111.0
15014,"[archaic, period, america, saw, chang, environ...",5,683.0,782.0,737.0,593.0,1407.0
15015,"[mobil, hunter-gather, individu, group, start,...",5,643.0,560.0,524.0,572.0,1149.0


In [ ]:
textos.describe()

In [ ]:
textos.info()

In [ ]:
textos.to_csv("Historia.csv")

In [ ]:
import pandas as pd
textos = pd.read_csv("Historia.csv")
textos = textos.iloc[:,1:]
textos

In [ ]:
textos.groupby(by=["Clase"]).count()

In [ ]:
textos.sample(10)

In [ ]:
len(vocabulario)

In [ ]:
len(frecuencias)

In [ ]:
clases_df

In [ ]:
save_obj(vocabulario, "Vocabulario")

In [ ]:
save_obj(frecuencias, "Frecuencias")

In [ ]:
save_obj(clases_df, "clases_df")

In [ ]:
textos.sample(10)

In [ ]:
int(np.argmax(textos.iloc[3337,-5:].values)) + 1

In [ ]:
int(textos.iloc[14317,1:2].values)

In [ ]:
int(np.argmax(textos.iloc[1621,-5:].values)) + 1 == int(textos.iloc[1621,1:2].values)

In [ ]:
textos.sample(10)

In [ ]:
i = 3
int(np.argmax(textos.iloc[i,-6:].values)) == int(textos.iloc[i,1:2].values)

In [ ]:
int(np.argmax(textos.iloc[i,-5:].values) + 1)

In [ ]:
textos.iloc[i,-6:].values

In [ ]:
textos["Clase1"][0]

In [ ]:
# textos = textos.iloc[:10,:]
textos

In [ ]:
textos["FreqOK"] = None
textos

In [ ]:
for i in textos.index:
    textos["FreqOK"][i] = int(np.argmax(textos.iloc[i,-6:-1].values)) + 1 == int(textos.iloc[i,1:2].values)
    # print((np.argmax(textos.iloc[i,-6:-1].values)) + 1 == int(textos.iloc[i,1:2].values))
textos    

In [ ]:
textos.sample(10)

In [ ]:
textos.sort_values(by=['FreqOK'])
textos

In [ ]:
textos.groupby(by=["FreqOK"]).count()

In [ ]:
textosF = textos[textos['FreqOK'] == False]
textosF

In [ ]:
textosF.groupby(by=["Clase"]).count()

In [ ]:
textosF1 = textosF[textosF['Clase'] == 1]
textosF1

In [ ]:
claseMax = np.argmax(textosF1.iloc[0,-6:-1].values) + 1
claseMax

In [ ]:
textosF1["ClasMax"] = None
textosF1

In [ ]:
indice = 0
for i in textosF1.index:
    textosF1["ClasMax"][i] = np.argmax(textosF1.iloc[indice,-7:-2].values) + 1
    indice +=1
textosF1.head(11)    

In [ ]:
textosF1.sample(10)    

In [ ]:
frecuencias = load_obj("Frecuencias")

In [ ]:
vocabulario = load_obj("Vocabulario")

In [ ]:
claseTrue = 1
clases = 5
palabrasConfusas = []
# indice = 0
totalPalabras = 0
for i in textosF1.index:
    """
    if indice > 1:
        break"""
    
    w = 0        
    for word in textosF1.loc[i,"Texto"].split():
        freqWord = []
        for k in range(clases + 1):
            freqWord.append(0)
        if w == 0:
            word = word[2:-2]
        else:   
            word = word[1:-2]
        w += 1                  
        if word in vocabulario:
            for k in range(1, len(freqWord)):
                print(word,"clase ",k,frecuencias[(word,k)])
                freqWord[k] = frecuencias[(word,k)]
        print(freqWord)    
        print(np.argmax(freqWord))
        print(np.argmax(freqWord) == claseTrue)        
        if np.argmax(freqWord) != claseTrue:
            palabrasConfusas.append(word)
        totalPalabras += w        
    indice += 1
print("palabras",totalPalabras)
print("confusas",len(palabrasConfusas))
palabrasConfusas

In [ ]:
claseTrue = 1
clases = 5

for word in palabrasConfusas:
    freqWord = []
    for k in range(clases + 1):
        freqWord.append(0)
    if word in vocabulario:
        print()                                  
        for i in range(1,6):
            print(word,"clase ",i,frecuencias[(word,i)]) 
            freqWord[i] = frecuencias[(word,i)]
        print(freqWord)    
        print(np.argmax(freqWord))
        print(np.argmax(freqWord) == claseTrue)
palabrasConfusas        

In [ ]:
if 'bronz' in vocabulario:
        print("está")

In [ ]:
print(frecuencias)

In [ ]:
len(palabrasConfusas)

In [ ]:
for word in palabrasConfusas:
    if word in vocabulario:
        print()                                  
        for i in range(1,6):
            print(word,"clase ",i,frecuencias[(word,i)]) 
            freqWord[i] = frecuencias[(word,i)]
        print(freqWord)    
        print(np.argmax(freqWord))
        print(np.argmax(freqWord) == claseTrue)
        print("Maximo", freqWord[np.argmax(freqWord)],"Clase",np.argmax(freqWord))
palabrasConfusas        

In [ ]:
confusas = pd.DataFrame(columns = ['Palabra' , 'Clase','FrecClase','ClaseMax','FrecMax'])

In [ ]:
for word in palabrasConfusas:
    if word in vocabulario: 
        freqWord = []
        for k in range(clases + 1):
            freqWord.append(0)
        for i in range(1,6):
            freqWord[i] = frecuencias[(word,i)]
        confusas = confusas.append({'Palabra' : word, 'Clase' : claseTrue ,'FrecClase' : freqWord[claseTrue], 'ClaseMax' : np.argmax(freqWord),'FrecMax' : freqWord[np.argmax(freqWord)] }, ignore_index = True)        
confusas

In [ ]:
confusas.to_csv("Palabras_Confusas.csv")

In [ ]:
import pandas as pd
confusas = pd.read_csv("Palabras_Confusas.csv")
confusas

In [ ]:
confusas = confusas.iloc[:,1:]
confusas

In [ ]:
confusas["Porcentaje"] = 100 * confusas["FrecClase"] / confusas["FrecMax"]
confusas

In [ ]:
confusas["Decenas"] = confusas["Porcentaje"] // 10
confusas

In [ ]:
confusas.groupby(by=["Decenas"]).count()

In [ ]:
confusas_clase1 = []
for i in confusas.index:
    if confusas["Decenas"][i] < 2:
        confusas_clase1.append(confusas["Palabra"][i])
len(confusas_clase1)        

In [ ]:
confusas_clase1

In [ ]:
frecuencias

In [ ]:
claseTrue = 1
clases = 5
cont = 0
for word in confusas_clase1:
    print()
    cont +=1
    print(word)
    for clase in range(1,clases + 1):
        print(frecuencias.get((word,clase),0))
        

In [ ]:
claseTrue = 1
clases = 5
frecuenciasMod = frecuencias.copy()
for word in confusas_clase1:
    frecuenciasMod[word,claseTrue] = 1

In [ ]:
claseTrue = 1
clases = 5
for word in confusas_clase1:
    print()
    print()
    print(word)
    for clase in range(1,clases + 1):
        print(frecuencias.get((word,clase),0), end = " ")
    print()    
    for clase in range(1,clases + 1):
        print(frecuenciasMod.get((word,clase),0), end = " ")